## Algorithme du recuit simulé

Dans cette partie, nous calculons une stratégie optimale en utilisant une algorithme de recuit simulé. Le principe de cet algorithme est inspiré de la métallurgie : on alterne des cycles de chauffage et de refroidissement afin de minimiser l'énergie du matériau.

Dans notre cas, le rôle de l'énergie est joué par le nombre de morts. On cherche les couples (n(t),η(t))  qui permettent de le minimiser. On a vu en utilisant le principe du maximum de Pontryagin (voir article) que le contrôle otpimal est de type bang bang. Les couples η, n sont donc à valeurs sur les bords du carré $C = [n_{inf},n_{sup}] × [\eta_{inf},\eta{sup}]$.

Pour simplifier, on suppose que les fonctions η et n sont constantes par morceaux. On discrétise le temps T de la simulation en $Ntcontrol$ sous intervalles, par exemple $Ntcontrol = 6$, et n et η sont constantes sur ces intervalles. Ainsi, une stratégie peut être représentée par un couple de vecteurs de tailles $Ntcontrol$.

L'algorithme de recuit simulé comporte deux étapes.

Premièrement, on modifie aléatoirement la stratégie courante. Pour ce faire, on prend un intervalle de temps aléatoire et une valeur aléatoire sur le bord du carré C. On remplace la valeurs de $(n,\eta)$ pour cet intervalle de temps par cette nouvelle valeur.

Deuxièmement, on calcule le nombre de morts pour cette nouvelle stratégie. S'il est inférieur à celui obtenu pour la stratégie précédente, on décide de le conserver. S'il est supérieure, on le conserve avec une probabilité $e^{-ΔE/T}$ où $T$ est la "température", un paramètre de l'algorithme qui décroit au cours des itérations. Lorsque la température tombe à 0, l'algorithme s'arrête.

Le fait d'accepter ainsi avec une certaine probabilité des stratégies augmentant l'énergie (le nombre de morts) permet en fait d'éviter de tomber dans un minimum local.

In [99]:
import simanneal
import numpy as np
import random as rd
import copy
import numpy as np
from scipy.ndimage import gaussian_filter
from math import *
import matplotlib.pyplot as plt

On commence par définir les paramètres, puis on redéfinit les fonctions nécessaires au calcul du nombre de mort pour chaque stratégie proposée pendant l'algorithme du recuit simulé.

In [100]:
# Paramètres

N = 30 # nombre de transmissibilités possibles
M = 40 # nombre de résistances possibles
R0max = 15 # taux de transmission maximal
gamma = 0.09 # taux de guérison
Beta = R0max * gamma * np.arange(N) / (N - 1) # transmissibilités possibles
Omega = np.arange(M) / (M - 1) # résistances possibles
mu = 0.02 # taux de mortalité
beta_i0 = 0.2 # taux de transmission initial
i0 = int(beta_i0 * (N - 1) / (gamma * R0max)) # indice de la transmissibilité initiale
j0 = 0 # indice de la résistance initiale
sigma = 1 # taux de mutation
C = 0.5 # taux de réinfection
Xi = np.zeros((N, M, N, M)) # matrice des réinfections
for j in range(M):
    for l in range(M):
        Xi[:, j, :, l] = C * max(Omega[l] - Omega[j], 0) / M

#T = 350 # durée de la simulation 
T = 350
dt = 0.1
#dt = 1

pop = 10**6 # population totale
h = 1 / pop

# Paramètres du contrôle
Nt_control = 6 # Discrétisation en temps du contrôle

def psi(x):
    if abs(x) < h:
        return 0
    elif abs(x) > 2 * h:
        return x
    elif x <= 2 * h  and x >= h:
        return -3 * (x**3)/(h**2) + 14 * (x**2) / h - 19 * x + 8 * h
    elif x <= -h and x >= -2 * h:
        return -3 * (x**3)/(h**2) - 14 * (x**2) / h - 19 * x - 8 * h

def Psi(X):
    return np.array([psi(e) for e in X.reshape(X.size)]).reshape(X.shape)

In [101]:
# Calcul des dérivées

def dS(S, I, n, eta):
    return -n - eta * np.dot(Beta, I).sum() * S

def dV(I, V, n, eta):
    return n - eta * Beta@I@Omega * V

def dI(S, I, V, R, eta):
    return eta * Beta[:, np.newaxis] * I * S \
        + eta * Beta[:, np.newaxis] * Omega * I * V \
        - mu * I - gamma * I \
        + eta * np.tensordot(Xi, R, 2) * I \
        + Psi(gaussian_filter(I, sigma, mode='constant', cval=0) - I)
    
def dR(I, R, eta):
    return gamma * I - eta * np.tensordot(np.transpose(Xi,(2,3,0,1)), R, 2) * I
    #return gamma * I - eta * np.tensordot(np.transpose(Xi,(2,3,0,1)), I, 2) * R

def dD(I):
    return mu * np.sum(I)

In [102]:
# Initialisation

def initialize():
    I0 = np.zeros((N, M)) # matrice des infections initiale
    I0[i0, j0] = 200 * 10**-5
    R0 = np.zeros((N, M)) # matrice des résistances initiale
    R0[i0, j0] = 0.25
    S0 = 1 - np.sum(I0) - np.sum(R0)
    V0 = 0
    D0 = 0


    return S0,I0,R0,V0,D0

In [103]:
def cost(U_n, U_eta):
    
    # Initialisation
    S,I,R,V,D = initialize()

    # Simulation

    S_l = np.zeros(int(T/dt))
    I_l = np.zeros((int(T/dt), N, M))
    R_l = np.zeros((int(T/dt), N, M))
    D_l = np.zeros(int(T/dt))
    V_l = np.zeros(int(T/dt))

    S_l = np.zeros(int(T/dt))
    I_l = np.zeros((int(T/dt), N, M))
    R_l = np.zeros((int(T/dt), N, M))
    D_l = np.zeros(int(T/dt))
    V_l = np.zeros(int(T/dt))

    for t in range(T):

        time_control = int(floor(Nt_control * t / T))
        n_t, eta_t = U_n[time_control], U_eta[time_control]

        S_l[t] = S
        I_l[t] = copy.deepcopy(I)
        R_l[t] = copy.deepcopy(R)
        D_l[t] = D
        V_l[t] = V

        val = min(n_t, S * (1 / dt - eta_t * np.sum(I.transpose() @ Beta)))
        # Newton
        Sp = dS(S, I, val, eta_t)
        Vp = dV(I, V, val, eta_t)
        Ip = dI(S, I, V, R, eta_t)
        Rp = dR(I, R, eta_t)
        Dp = dD(I)
        
        S = S + dt * Sp
        V = V + dt * Vp
        I = I + dt * Ip
        R = R + dt * Rp
        D = D + dt * Dp

    return D
    

La solution optimale étant localisée sur le carré, on discrétise celui-ci. On représente les différents couples possibles par un seul vecteur. La fonction indice_to_couple permet d'associer un élément de ce vecteur aux valeurs correspondantes de (n, η).

In [104]:
# square discretization

N_n = 5 # discrétisation de n
N_eta = 5 # discrétisation de eta

eta_inf = 1.
eta_sup = 1.
n_inf = 0.
n_sup = 0.006

eta_vec = np.linspace(eta_inf, eta_sup, N_eta)
n_vec = np.linspace(n_inf, n_sup, N_n)

if eta_sup == eta_inf:
    N_eta = 0
    eta_vec = []

if n_inf == n_sup:
    N_n = 0
    n_vec = []

nb_states = 2*(N_n + N_eta - (N_n != 0) - (N_eta != 0)) # On élimine les doublons

possible_states = [i for i in range(nb_states)]

# On les range dans cet ordre : [n,eta_inf],[nsup,eta], [n,eta_sup],[n_inf,eta]

def indice_to_couple(i, N_n, N_eta, eta_vec, n_vec):
    if i < N_n:
        return n_vec[i], eta_inf
    elif i < N_n + N_eta - 1:
        return n_sup, eta_vec[i - N_n + 1]
    elif i < N_n + N_eta + N_n - 2:
        return n_vec[N_n + N_eta + N_n - 3 - i], eta_sup
    else:
        return n_inf, eta_vec[2*(N_n + N_eta - 2) - i]

On définit ensuite la manière de calculer l'énergie pour une stratégie (grâce à la fonction coût présentée plus haut), ainsi que la façon de modifier une stratégie (de manière aléatoire, comme expliqué plus haut).

In [105]:
class OptimalVaccineStrategy(simanneal.Annealer):

    def move(self):

        # Choix d'un point au hasard sur le carré
        k = rd.randint(0, nb_states - 1)

        # Choix d'un indice temporel pour lequel on va remplace le contrôle par le point précédent
        t_rand = rd.randint(0,Nt_control -1)

        self.state[t_rand], self.state[t_rand + Nt_control] = indice_to_couple(k, N_n, N_eta, eta_vec, n_vec)
    
    def energy(self):
        
        U_n = self.state[:Nt_control]
        U_eta = self.state[Nt_control:]

        return cost(U_n, U_eta)

In [106]:
initial_state = np.array([n_sup]*Nt_control + [eta_inf]*Nt_control)
ovs = OptimalVaccineStrategy(initial_state)
ovs.steps = 5000
params, nb_deaths = ovs.anneal()

n_opt, eta_opt = params[:Nt_control], params[Nt_control:]

print("Nombre de décès : ", nb_deaths)

plt.plot(n_opt)
plt.plot(eta_opt)
plt.show()

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
  3613.59943          0.00   100.00%    38.00%     0:20:18     1:16:22